In [1]:



# Install pip3-autoremove if not already installed
!pip install pip3-autoremove

# Uninstall old Torch and related packages
!pip-autoremove torch torchvision torchaudio -y

# Install Torch, TorchVision, and TorchAudio with CUDA 12.1
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121 --quiet

# Install additional required libraries
!pip install unsloth datasets trl --quiet


pyarrow 18.1.0 is installed but pyarrow<15.0.0a0,>=14.0.1 is required
Redoing requirement with just package name...
dask 2024.12.1 is installed but dask==2024.8.0 is required
Redoing requirement with just package name...
google-api-core 1.34.1 is installed but google-api-core[grpc]<3.0.0dev,>=2.16.0 is required
Redoing requirement with just package name...
notebook 6.5.4 is installed but notebook==6.5.5 is required
Redoing requirement with just package name...
pyarrow 18.1.0 is installed but pyarrow<16,>=2 is required
Redoing requirement with just package name...
tbb 2022.0.0 is installed but tbb==2021.* is required
Redoing requirement with just package name...
google-api-core 1.34.1 is installed but google-api-core<3.0.0dev,>=2.10.2 is required
Redoing requirement with just package name...
The 'pycairo>=1.16.0' distribution was not found and is required by the application
Skipping pycairo
torchvision 0.19.1+cu121 (/usr/local/lib/python3.10/dist-packages)
    torch 2.4.1+cu121 (/usr/lo

In [2]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from peft import PeftModel
import torch

# Step 1: Load the base model (Gemma 2)
base_model_name = "google/gemma-2-9b-it"  # Specify the base model name
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_name,
    max_seq_length=2048,
    dtype=None,  # Auto-detect hardware
    load_in_4bit=True,
)

# Step 2: Load the LoRA adapter
adapter_path = "/kaggle/input/gemma2-9b_english-bengali/pytorch/default/1/Gemma2_BanglaEnglish"  # Replace with your adapter directory
# Use PeftModel to load the LoRA adapter
model = PeftModel.from_pretrained(base_model, adapter_path)

# Step 3: Move the model to the appropriate device (GPU or CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Step 4: Define the translation prompt
translation_prompt = """Below is a task instruction paired with input text. Your job is to provide an accurate translation.

### Task:
{}

### Input Text:
{}

### Translation:
{}"""

FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
inputs = tokenizer(
[
    translation_prompt.format(
        "Translate to English", # instruction
        " জিম্বাবুয়েকে হারাল আফগানিস্তান", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.12: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

['<bos>Below is a task instruction paired with input text. Your job is to provide an accurate translation.\n\n### Task:\nTranslate to English\n\n### Input Text:\n জিম্বাবুয়েকে হারাল আফগানিস্তান\n\n### Translation:\nafghanistan beat zimbabwe .<eos>']